In [2]:
import simcat   
import toytree    ## requires github Master branch currently
import random
import numpy as np
from copy import deepcopy
import toyplot

In [144]:
def sample_branches(toytreeobj, betaval = 1,returndict = None):
    """
    Takes an input topology and samples branch lengths
    
    Parameters:
    -----------
    toytreeobj: toytree
        The topology for which we want to generate branch lengths
    betaval: int/float (default=1)
        The value of beta for the exponential distribution used to generate
        branch lengths. This is inverse of rate parameter.
    returndict: str (default=None)
        If "only", returns only a dictionary matching nodes to heights.
        If "both", returns toytree and dictionary as a tuple.
    """
    tree=deepcopy(toytreeobj)
    def set_node_height(node,height):
        childn=node.get_children()
        for child in childn:
            child.dist=height - child.height
    testobj = []
    
    # we'll just append each step on each chain to these lists to keep track of where we've already been
    allnodes = []
    allnodeheights = []
    
    # d is deprecated (it was being returned) and stores the values for each chain
    #d = {}
    
    ## testobj is our traversed full tree
    for i in tree.tree.traverse():
        testobj.append(i)
    
    # start by getting branch lengths for the longest branch
    longbranch=sum([testobj[-1] in i for i in testobj]) # counts the number of subtrees containing last leaf, giving length of longest branch
    longbranchnodes = np.random.exponential(betaval,longbranch) # segment lengths of longest branch
    longbranchnodes[0] = np.random.uniform(low=0.0,high=longbranchnodes[0]) # cut the edge to the leaf with a uniform draw
    
    # get the heights of nodes along this longest (most nodes) branch
    nodeheights = []
    currheight = 0
    for i in longbranchnodes:
        currheight += i
        nodeheights.append(currheight)
    nodeheights = nodeheights[::-1]

    # get indices to accompany long chain
    lcidx = []
    for i in testobj:
        if testobj[-1] in i.get_leaves():
            lcidx.append(i.idx)
    #d['0heights'] = np.array(nodeheights)
    #d['0nodes'] = np.array(lcidx[:-1])
    
    allnodes = allnodes + lcidx[:-1]
    allnodeheights = allnodeheights + nodeheights
    
    # get other necessary chains to parse
    other_chains = []
    for i in range(len(testobj)-1)[::2]:
        if len(testobj[i+1].get_leaves()) > 1:
            other_chains.append(testobj[i+1])

    # now solve
    for chainnum in range(len(other_chains)): # parse the remaining chains one at a time
        otr=other_chains[chainnum]
        # find where this chain connects to the a chain we've already solved
        firstancestor = otr.get_ancestors()[0].idx
        # which nodeheight does this branch from
        paridx=np.argmax(np.array(allnodes) == firstancestor) 
        
        # traverse the new 
        testobj1 = []
        nodes = []
        
        # save a list of nodes
        for i in otr.traverse():
            testobj1.append(i)
        
        # save the nodes that include the end of the chain (because branches out to other chains might not)
        for i in testobj1:
            if testobj1[-1] in i.get_leaves():
                nodes.append(i.idx)
        
        # make node index list to accompany lengths
        lennodes= nodes[:-1] # don't save ending leaf index
        lennodes.insert(0,firstancestor) # make chain list start with ancestor
        
        # figure out how many exponential draws to make for this chain (i.e. # new nodes + 1)
        num_new_branches = sum([testobj1[-1] in i for i in testobj1])+1
        
        # initialize array to hold the draws
        mir_lens = np.zeros((sum([testobj1[-1] in i for i in testobj1])+1))
        # draw until we have a new set of exponential branch lengths that fit the constraints of our tree height
        while not (sum(mir_lens[:(len(mir_lens)-1)]) < allnodeheights[paridx] and (sum(mir_lens) > allnodeheights[paridx])):
            mir_lens = np.random.exponential(betaval,num_new_branches) ## length of longest branch
        
        # now let's save each node value as a height
        mir_lens_heights = np.zeros((len(mir_lens)))
        subsum = 0
        for i in range(len(mir_lens)):
            mir_lens_heights[i] = allnodeheights[paridx] - subsum
            subsum = subsum + mir_lens[i]
            
        # add our new node indices with their heights to the full list
        allnodes = list(allnodes) + list(lennodes)
        allnodeheights = list(allnodeheights) + list(mir_lens_heights)
        
        #d[(str(chainnum+1)+"heights")] = mir_lens_heights
        #d[(str(chainnum+1)+"nodes")] = np.array(lennodes)
    
    # make a final dictionary of node heights, eliminating redundancy
    n = dict(set(zip(*[allnodes,allnodeheights])))
    
    if returndict == "only":
        return n #d
    elif returndict == "both":
        # create the tree object
        for leaf in tree.tree.get_leaves():
            for node in leaf.iter_ancestors():
                set_node_height(node,n[node.idx])
        return (tree,n)
    else:
        # create the tree object
        for leaf in tree.tree.get_leaves():
            for node in leaf.iter_ancestors():
                set_node_height(node,n[node.idx])
        return tree

In [48]:
## generate a random tree
random.seed(123)
tree = toytree.rtree(10)
tree.tree.convert_to_ultrametric()
tree.draw(tree_style='c', node_labels='idx', tip_labels=False, padding=50);

idx: 0
name: t-9
dist: 1
support: 100
height: 0 0 idx: 1
name: t-8
dist: 1
support: 100
height: 0 1 idx: 2
name: t-7
dist: 2
support: 100
height: 0 2 idx: 3
name: t-6
dist: 1
support: 100
height: 0 3 idx: 4
name: t-5
dist: 1
support: 100
height: 0 4 idx: 5
name: t-4
dist: 2
support: 100
height: 0 5 idx: 6
name: t-3
dist: 1
support: 100
height: 0 6 idx: 7
name: t-2
dist: 1
support: 100
height: 0 7 idx: 8
name: t-1
dist: 2
support: 100
height: 0 8 idx: 9
name: t-0
dist: 3
support: 100
height: 0 9 idx: 10
name: i10
dist: 1
support: 1
height: 1 10 idx: 13
name: i13
dist: 1
support: 1
height: 2 14 idx: 11
name: i11
dist: 1
support: 1
height: 1 13 idx: 14
name: i14
dist: 1
support: 1
height: 2 16 idx: 16
name: i16
dist: 1
support: 1
height: 3 15 idx: 12
name: i12
dist: 1
support: 1
height: 1 11 idx: 15
name: i15
dist: 1
support: 1
height: 2 12 idx: 17
name: i17
dist: 1
support: 1
height: 3 17 idx: 18
name: i18
dist: 0
support: 100
height: 4 18

### We can now sample new branch lengths for this topology, returning a toytree object:

In [49]:
sample_branches(tree)

### Or a dictionary of node heights:

In [50]:
sample_branches(tree,returndict="only",betaval=9)

{10: 0.59299200292740617,
 11: 0.24990084436810012,
 12: 3.7828557796223876,
 13: 3.5157302853956351,
 14: 2.5472608962648775,
 15: 5.2079711040570515,
 16: 6.2504853127316462,
 17: 5.8851692024594797,
 18: 9.7345010742787998}

### Do this several times:

In [51]:
ntrees = 4
canvas = toyplot.Canvas(width=800, height=200)
axes = [canvas.cartesian(grid=(1, ntrees, idx)) for idx in range(ntrees)]
for i in range(ntrees):
    ax = axes[i]
    ax.show = False
    sample_branches(tree).draw(
        axes=ax, tree_style='c', node_labels='name', tip_labels=False, node_size=16)

idx: 0
name: t-9
dist: 0.161658121156
support: 100
height: 0.0 t-9 idx: 1
name: t-8
dist: 0.161658121156
support: 100
height: 0.0 t-8 idx: 2
name: t-7
dist: 0.282599463046
support: 100
height: 0.0 t-7 idx: 3
name: t-6
dist: 0.132483574805
support: 100
height: 0.0 t-6 idx: 4
name: t-5
dist: 0.132483574805
support: 100
height: 0.0 t-5 idx: 5
name: t-4
dist: 0.385496118526
support: 100
height: 0.0 t-4 idx: 6
name: t-3
dist: 0.210556862122
support: 100
height: 0.0 t-3 idx: 7
name: t-2
dist: 0.210556862122
support: 100
height: 0.0 t-2 idx: 8
name: t-1
dist: 0.418644895403
support: 100
height: 0.0 t-1 idx: 9
name: t-0
dist: 0.612553813184
support: 100
height: 0.0 t-0 idx: 10
name: i10
dist: 0.12094134189
support: 1
height: 0.161658121156 i10 idx: 13
name: i13
dist: 0.268894353213
support: 1
height: 0.282599463046 i14 idx: 11
name: i11
dist: 0.253012543721
support: 1
height: 0.132483574805 i13 idx: 14
name: i14
dist: 0.165997697733
support: 1
height: 0.385496118526 i16 idx: 16
name: i16
dist: 0.269244287182
support: 1
height: 0.551493816259 i15 idx: 12
name: i12
dist: 0.208088033281
support: 1
height: 0.210556862122 i11 idx: 15
name: i15
dist: 0.193908917781
support: 1
height: 0.418644895403 i12 idx: 17
name: i17
dist: 0.208184290257
support: 1
height: 0.612553813184 i17 idx: 18
name: i18
dist: 0.0
support: 100
height: 0.820738103441 i18 idx: 0
name: t-9
dist: 0.588740060585
support: 100
height: 0.0 t-9 idx: 1
name: t-8
dist: 0.588740060585
support: 100
height: 0.0 t-8 idx: 2
name: t-7
dist: 3.09180886637
support: 100
height: 0.0 t-7 idx: 3
name: t-6
dist: 0.91711326375
support: 100
height: 0.0 t-6 idx: 4
name: t-5
dist: 0.91711326375
support: 100
height: 0.0 t-5 idx: 5
name: t-4
dist: 1.15616906653
support: 100
height: 0.0 t-4 idx: 6
name: t-3
dist: 1.74869067099
support: 100
height: 0.0 t-3 idx: 7
name: t-2
dist: 1.74869067099
support: 100
height: 0.0 t-2 idx: 8
name: t-1
dist: 2.57411294018
support: 100
height: 0.0 t-1 idx: 9
name: t-0
dist: 2.88825020621
support: 100
height: 0.0 t-0 idx: 10
name: i10
dist: 2.50306880579
support: 1
height: 0.588740060585 i10 idx: 13
name: i13
dist: 0.0923544825308
support: 1
height: 3.09180886637 i14 idx: 11
name: i11
dist: 0.239055802776
support: 1
height: 0.91711326375 i13 idx: 14
name: i14
dist: 2.02799428238
support: 1
height: 1.15616906653 i16 idx: 16
name: i16
dist: 3.28579904145
support: 1
height: 3.18416334891 i15 idx: 12
name: i12
dist: 0.825422269193
support: 1
height: 1.74869067099 i11 idx: 15
name: i15
dist: 0.314137266029
support: 1
height: 2.57411294018 i12 idx: 17
name: i17
dist: 3.58171218414
support: 1
height: 2.88825020621 i17 idx: 18
name: i18
dist: 0.0
support: 100
height: 6.46996239035 i18 idx: 0
name: t-9
dist: 0.205180983722
support: 100
height: 0.0 t-9 idx: 1
name: t-8
dist: 0.205180983722
support: 100
height: 0.0 t-8 idx: 2
name: t-7
dist: 1.64693002221
support: 100
height: 0.0 t-7 idx: 3
name: t-6
dist: 0.55685759413
support: 100
height: 0.0 t-6 idx: 4
name: t-5
dist: 0.55685759413
support: 100
height: 0.0 t-5 idx: 5
name: t-4
dist: 1.67658752745
support: 100
height: 0.0 t-4 idx: 6
name: t-3
dist: 2.10666356881
support: 100
height: 0.0 t-3 idx: 7
name: t-2
dist: 2.10666356881
support: 100
height: 0.0 t-2 idx: 8
name: t-1
dist: 2.63599358714
support: 100
height: 0.0 t-1 idx: 9
name: t-0
dist: 2.82935771155
support: 100
height: 0.0 t-0 idx: 10
name: i10
dist: 1.44174903849
support: 1
height: 0.205180983722 i10 idx: 13
name: i13
dist: 1.72044098016
support: 1
height: 1.64693002221 i14 idx: 11
name: i11
dist: 1.11972993332
support: 1
height: 0.55685759413 i13 idx: 14
name: i14
dist: 1.69078347492
support: 1
height: 1.67658752745 i16 idx: 16
name: i16
dist: 0.224096731618
support: 1
height: 3.36737100237 i15 idx: 12
name: i12
dist: 0.529330018334
support: 1
height: 2.10666356881 i11 idx: 15
name: i15
dist: 0.193364124405
support: 1
height: 2.63599358714 i12 idx: 17
name: i17
dist: 0.762110022442
support: 1
height: 2.82935771155 i17 idx: 18
name: i18
dist: 0.0
support: 100
heig

### Currently, the toytree object part breaks when the number of taxa gets above 16 -- the tips no longer line up.

In [52]:
random.seed(123)
tree = toytree.rtree(16)
tree.tree.convert_to_ultrametric()
sample_branches(tree).draw(tree_style='c', node_labels='idx', tip_labels=False, padding=50);

idx: 0
name: t-15
dist: 0.217502753765
support: 100
height: 0.0 0 idx: 1
name: t-14
dist: 0.217502753765
support: 100
height: 0.0 1 idx: 2
name: t-13
dist: 0.0184083151939
support: 100
height: 0.0 2 idx: 3
name: t-12
dist: 0.0184083151939
support: 100
height: 0.0 3 idx: 4
name: t-11
dist: 2.8154621103
support: 100
height: 0.0 4 idx: 5
name: t-10
dist: 2.8154621103
support: 100
height: 0.0 5 idx: 6
name: t-9
dist: 0.0632568136081
support: 100
height: 0.0 6 idx: 7
name: t-8
dist: 0.0632568136081
support: 100
height: 0.0 7 idx: 8
name: t-7
dist: 1.05965640972
support: 100
height: 0.0 8 idx: 9
name: t-6
dist: 2.03951605584
support: 100
height: 0.0 9 idx: 10
name: t-5
dist: 2.03951605584
support: 100
height: 0.0 10 idx: 11
name: t-4
dist: 2.5706452978
support: 100
height: 0.0 11 idx: 12
name: t-3
dist: 2.5706452978
support: 100
height: 0.0 12 idx: 13
name: t-2
dist: 2.92656332611
support: 100
height: 0.0 13 idx: 14
name: t-1
dist: 3.68024237106
support: 100
height: 0.0 14 idx: 15
name: t-0
dist: 3.68024237106
support: 100
height: 0.0 15 idx: 16
name: i16
dist: 1.2239288939
support: 1
height: 0.217502753765 16 idx: 17
name: i17
dist: 1.42302333247
support: 1
height: 0.0184083151939 17 idx: 19
name: i19
dist: 1.67061503488
support: 1
height: 1.44143164766 20 idx: 20
name: i20
dist: 0.296584572244
support: 1
height: 2.8154621103 24 idx: 24
name: i24
dist: 1.37458676561
support: 1
height: 3.11204668255 25 idx: 18
name: i18
dist: 0.996399596107
support: 1
height: 0.0632568136081 19 idx: 21
name: i21
dist: 2.40240637138
support: 1
height: 1.05965640972 18 idx: 22
name: i22
dist: 1.42254672526
support: 1
height: 2.03951605584 21 idx: 25
name: i25
dist: 1.02457066706
support: 1
height: 3.4620627811 28 idx: 28
name: i28
dist: 0.288838933278
support: 1
height: 4.48663344816 27 idx: 23
name: i23
dist: 0.355918028309
support: 1
height: 2.5706452978 22 idx: 26
name: i26
dist: 0.780013350882
support: 1
height: 2.92656332611 23 idx: 27
name: i27
dist: 0.0263343059295
support: 1
height: 3.68024237106 26 idx: 29
name: i29
dist: 1.06889570444
support: 1
height: 3.70657667699 29 idx: 30
name: i30
dist: 0.0
support: 100
height: 4.77547238144 30

In [119]:
random.seed(123)
tree = toytree.rtree(17)
tree.tree.convert_to_ultrametric()
samptree, sampdict = sample_branches(tree,returndict="both")
print("Top node height should be: "+str(sampdict[32]))
print("\n" + "Top node height is instead: " + str(samptree.tree.search_nodes(idx = 32)[0].height))

Top node height should be: 4.22325539801

Top node height is instead: 4.22325539801


### So something is off. Good news is that the dictionary itself seems to be correct.

### The other good news is that this still works relatively quickly for big trees, although some of them then take a very long time. Might be good to build in a "bailout" option.

In [11]:
%%timeit
random.seed(123)
tree = toytree.rtree(70)
tree.tree.convert_to_ultrametric()
print(len(sample_branches(tree))) ## 68 internal nodes!

68
68
68
68
1 loop, best of 3: 137 ms per loop


In [146]:
%%timeit
random.seed(123)
tree = toytree.rtree(20)
tree.tree.convert_to_ultrametric()
for i in range(1000):
     sample_branches(tree)

1 loop, best of 3: 46.4 s per loop
